In [1]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification,TrainingArguments,Trainer,AutoConfig

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion", problem_type="multi_label_classification")


2024-12-02 15:50:10.928205: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 15:50:10.952614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd


In [5]:
df = pd.read_csv("data/BERT_training_data2.csv")
df

,text,emotions
0,i feel awful about it too because it s my job ...,sadness
1,im alone i feel awful,sadness
2,ive probably mentioned this before but i reall...,joy
3,i was feeling a little low few days back,sadness
4,i am one of those people who feels like going ...,joy
...,...,...
367278,that was what i felt when i was finally accept...,joy
367279,i take every day as it comes i m just focussin...,fear
367280,i just suddenly feel that everything was fake,sadness
367281,im feeling more eager than ever to claw back w...,joy


In [4]:
df = pd.read_csv("data/BERT_training_data2.csv")
training_set = df[:int(len(df)/5*4)]
training_set

,text,emotions
0,i feel awful about it too because it s my job ...,sadness
1,im alone i feel awful,sadness
2,ive probably mentioned this before but i reall...,joy
3,i was feeling a little low few days back,sadness
4,i am one of those people who feels like going ...,joy
...,...,...
293821,i feel like dom s rants and violent outburst c...,anger
293822,i hear from women who feel intimidated by clot...,fear
293823,id been feeling smug that we didnt have it rou...,joy
293824,i feel like i m being tricked for a numb,sadness


In [8]:
inputs = tokenizer("I hate that", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

In [9]:
logits

tensor([[ 3.4470, -1.1727, -0.9133, -0.6030]])

In [10]:
predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]


In [11]:
num_labels = len(model.config.id2label)
num_labels

4

In [12]:
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion", num_labels=num_labels, problem_type="multi_label_classification"
)

In [13]:
labels = torch.sum(
    torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
).to(torch.float)
labels

tensor([[1., 0., 0., 0.]])

In [14]:
loss = model(**inputs, labels=labels).loss
loss

tensor(0.2687, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)